<a href="https://colab.research.google.com/github/Natural-Language-Processing-YU/M3_Assignment/blob/main/scripts/m3_assignment_part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [ ]:
import nltk
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Sequential, Model



In [ ]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [ ]:
# Import the NLTK library and access its corpus module, specifically the names corpus
names = nltk.corpus.names

# Retrieve the file identifiers for the names corpus
names.fileids()

# Retrieve a list of male names from the names corpus
male_names = names.words('male.txt')

# Retrieve a list of female names from the names corpus
female_names = names.words('female.txt')

# Find any names that appear in both the male and female name lists
k=[w for w in male_names if w in female_names]

# Print the common names found in both the male and female name lists
print(k)


['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis', 'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel', 'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', 'Barrie', 'Barry', 'Beau', 'Bennie', 'Benny', 'Bernie', 'Bert', 'Bertie', 'Bill', 'Billie', 'Billy', 'Blair', 'Blake', 'Bo', 'Bobbie', 'Bobby', 'Brandy', 'Brett', 'Britt', 'Brook', 'Brooke', 'Brooks', 'Bryn', 'Cal', 'Cam', 'Cammy', 'Carey', 'Carlie', 'Carlin', 'Carmine', 'Carroll', 'Cary', 'Caryl', 'Casey', 'Cass', 'Cat', 'Cecil', 'Chad', 'Chris', 'Chrissy', 'Christian', 'Christie', 'Christy', 'Clair', 'Claire', 'Clare', 'Claude', 'Clem', 'Clemmie', 'Cody', 'Connie', 'Constantine', 'Corey', 'Corrie', 'Cory', 'Courtney', 'Cris', 'Daffy', 'Dale', 'Dallas', 'Dana', 'Dani', 'Daniel', 'Dannie', 'Danny', 'Darby', 'Darcy', 'Darryl', 'Daryl', 'Deane', 'Del', 'Dell', 'Demetris', 'Dennie', 'Denny', 'Devin', 'Devon', 'Dion', 'Dionis', 'Dominique', 'Donnie', 'Donny', 'Do

In [ ]:
# Combine the lists of male and female names into one list
all_names = male_names + female_names

# Create labels: 0 for male names and 1 for female names, using the length of male and female names lists
labels = [0] * len(male_names) + [1] * len(female_names)

# Split the combined names and labels into training and testing sets, with 20% for testing
train_names, test_names, train_labels, test_labels = train_test_split(all_names, labels, test_size=0.2, random_state=42)

# Initialize the BERT tokenizer from the pre-trained 'bert-base-uncased' model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the training and testing names sequences using the BERT tokenizer
train_sequences = tokenizer(train_names, truncation=True, padding=True, return_tensors='tf')
test_sequences = tokenizer(test_names, truncation=True, padding=True, return_tensors='tf')

# Convert the labels to NumPy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Build the model using a Sequential API
model = Sequential()

# Add an Embedding layer to represent the input words, with a vocabulary size and embedding dimension of 64
model.add(Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=64, input_length=train_sequences.input_ids.shape[1]))

# Add an LSTM layer with 64 units to process the embedded sequences
model.add(LSTM(64))

# Add a Dense layer with one unit and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer, binary crossentropy loss, and accuracy metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training sequences with their corresponding labels, using 10 epochs and a batch size of 32
model.fit(train_sequences.input_ids, train_labels, epochs=10, batch_size=32, validation_data=(test_sequences.input_ids, test_labels))

# Evaluate the trained model on the testing sequences and labels
loss, accuracy = model.evaluate(test_sequences.input_ids, test_labels)

# Print the test accuracy
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
199/199 [==============================] - 12s 47ms/step - loss: 0.5665 - accuracy: 0.6952 - val_loss: 0.4912 - val_accuracy: 0.7445
Epoch 2/10
199/199 [==============================] - 3s 13ms/step - loss: 0.3401 - accuracy: 0.8516 - val_loss: 0.5490 - val_accuracy: 0.7565
Epoch 3/10
199/199 [==============================] - 3s 15ms/step - loss: 0.2088 - accuracy: 0.9078 - val_loss: 0.5448 - val_accuracy: 0.7533
Epoch 4/10
199/199 [==============================] - 5s 25ms/step - loss: 0.1719 - accuracy: 0.9185 - val_loss: 0.6704 - val_accuracy: 0.7401
Epoch 5/10
199/199 [==============================] - 4s 18ms/step - loss: 0.1499 - accuracy: 0.9249 - val_loss: 0.7898 - val_accuracy: 0.7275
Epoch 6/10
199/199 [==============================] - 2s 11ms/step - loss: 0.1323 - accuracy: 0.9300 - val_loss: 0.9030 - val_accuracy: 0.7476
Epoch 7/10
199/199 [==============================] - 2s 11ms/step - loss: 0.1191 - accuracy: 0.9327 - val_loss: 0.9308 - val_accuracy: 0.745

In [ ]:
# Convert the training and testing sequences to NumPy arrays
train_input_ids = train_sequences['input_ids'].numpy()
train_attention_mask = train_sequences['attention_mask'].numpy()
train_token_type_ids = train_sequences['token_type_ids'].numpy()

test_input_ids = test_sequences['input_ids'].numpy()
test_attention_mask = test_sequences['attention_mask'].numpy()
test_token_type_ids = test_sequences['token_type_ids'].numpy()

# Convert the labels to NumPy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Define input layers for input_ids, attention_mask, and token_type_ids
input_ids = Input(shape=(train_input_ids.shape[1],), dtype=tf.int32)
attention_mask = Input(shape=(train_attention_mask.shape[1],), dtype=tf.int32)
token_type_ids = Input(shape=(train_token_type_ids.shape[1],), dtype=tf.int32)

# Load pre-trained BERT model for sequence classification
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Connect the input layers to the BERT model
outputs = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

# Build the model using the BERT model outputs
model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=outputs.logits)

# Compile the model with Adam optimizer, sparse categorical crossentropy loss, and accuracy metric
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model on the training data with batch size of 32 and 10 epochs
model.fit([train_input_ids, train_attention_mask, train_token_type_ids], train_labels, batch_size=32, epochs=10)

# Evaluate the trained model on the testing data
loss, accuracy = model.evaluate([test_input_ids, test_attention_mask, test_token_type_ids], test_labels)

# Print the test accuracy
print(f"Test Accuracy: {accuracy:.4f}")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 7)]          0           []                               
                                                                                                  
 input_17 (InputLayer)          [(None, 7)]          0           []                               
                                                                                                  
 input_18 (InputLayer)          [(None, 7)]          0           []                               
                                                                                                  
 tf_bert_for_sequence_classific  TFSequenceClassifie  109483778  ['input_16[0][0]',               
 ation_5 (TFBertForSequenceClas  rOutput(loss=None,               'input_17[0][0]',         

# References
1. https://arxiv.org/pdf/2102.03692.pdf
2. https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/exercise/13-attention.html
3. https://towardsdatascience.com/deep-learning-gender-from-name-lstm-recurrent-neural-networks-448d64553044
4. https://www.nltk.org/book/ch02.html#sec-lexical-resources